# Trace Buffer - Tracing IIC Transactions
The Trace_Buffer class can monitor the waveform and transations on PMODA, PMODB, and ARDUINO connectors.

This demo shows how to use this class to track IIC transactions. For this demo, users have to connect the Pmod TMP2 sensor to PMODA.

## Step 1: Overlay Management
Users have to import all the necessary classes. Make sure to use the right bitstream.

In [1]:
from pprint import pprint
from time import sleep
from pynq import PL
from pynq import Overlay
from pynq.drivers import Trace_Buffer
from pynq.iop import Pmod_TMP2
from pynq.iop import PMODA
from pynq.iop import PMODB
from pynq.iop import ARDUINO

ol = Overlay("base.bit")
ol.download()

## Step 2: Instantiating Temperature Sensor
Although this demo can also be done on PMODB, we use PMODA in this demo.

Set the log interval to be 1ms. This means the IO Processor (IOP) will read temperature values every 1ms.

In [2]:
tmp2 = Pmod_TMP2(PMODA)
tmp2.set_log_interval_ms(1)

## Step 3: Tracking Transactions
Instantiating the trace buffer with IIC protocol. The sample rate is set to 1MHz. Although the IIC clock is only 100kHz, we still have to use higher sample rate to keep track of IIC control signals from IOP.

After starting the trace buffer DMA, also start to issue IIC reads for 1 second. Then stop the trace buffer DMA.

Users can also use the following code for initialization when no decoder is available:
```python
tr_buf = Trace_Buffer(PMODA,pins=[2,3],protocol="i2c",rate=1000000)
```

In [3]:
tr_buf = Trace_Buffer(PMODA,pins=[2,3],probes=['SCL','SDA'],
                      protocol="i2c",rate=1000000)

# Start the trace buffer
tr_buf.start()

# Issue reads for 1 second
tmp2.start_log()
sleep(1)
tmp2_log = tmp2.get_log()

# Stop the trace buffer
tr_buf.stop()

## Step 4: Parsing and Decoding Transactions
The trace buffer object is able to parse the transactions into a `*.csv` file (saved into the same folder as this script).
    
Then the trace buffer object can also decode the transactions using the open-source `sigrok` decoders. The decoded file (`*.pd`) is saved into the same folder as this script.

Users will not be able to use the `decode()` function when no decoder is available. In that case, just delete the last line in the following cell.

Reference:
https://sigrok.org/wiki/Main_Page

In [4]:
# Set up samples
start = 500
stop = 3500

# Parsing and decoding samples
tr_buf.parse("i2c_trace.csv",start,stop)
tr_buf.decode("i2c_trace.pd")

## Step 5: Displaying the Result

The final waveform and decoded transactions are shown using the open-source `wavedrom` library. All the parsed and decoded samples will be displayed.

**Note**: It may take a while for the waveforms to be displayed.

Reference:
https://www.npmjs.com/package/wavedrom

In [5]:
tr_buf.display()